In [2]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [5]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nHome\n\nLocal removalists\n\nLocal removalists\n\nInterstate removalists\n\nSydney interstate removalists\n\nMelbourne interstate removalists\n\nBrisbane interstate removalists\n\nAdelaide interstate removalists\n\nCanberra interstate removalists\n\nContact\n\nGet a quote\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle Rating\n\n5 stars, 111 reviews\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\r\n                                Victoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Vic

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  15


In [9]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nHome\n\nLocal removalists\n\nLocal removalists\n\nInterstate removalists\n\nSydney interstate removalists\n\nMelbourne interstate removalists\n\nBrisbane interstate removalists\n\nAdelaide interstate removalists\n\nCanberra interstate removalists\n\nContact\n\nGet a quote\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle Rating\n\n5 stars, 111 reviews\n\nRequst A call for You:')

In [11]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What kind of services they provide?")


In [13]:
len(retrieved_docs)

6

In [14]:
print(retrieved_docs[0].page_content)

Apartment Moving
                                Efficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.

Villa Moving
                                 Comprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.

Household Moving
                                Full-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.

Office Moving
                                Specialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.

Furniture Moving
                                Experienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.


In [15]:
llm = OpenAI(temperature=0.4, max_tokens=500)

In [16]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [17]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [18]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])



System: Victoria on Move provides local and interstate removal services for residential and commercial properties. They also offer packing and unpacking services, as well as customized moving plans to fit specific requirements and schedules. Their team of experienced professionals ensures a stress-free and seamless moving experience for their customers.
